# <span style="text-decoration: underline; color: #ffff00">March Tabular Playground Series competition</span> 
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTJ9fIwYofhSehmxC8QvHDXMU8AY9VP8DvcdA&usqp=CAU)
## <span style='color:#ffd700'> Importación de librerías </span>

In [1]:
import zipfile
import os
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import xgboost

## <span style='color:#ffd700'> Descarga de información y generación de DataFrames </span>
Descargo la información directo desde la API de Kaggle, la descomprimo y borro el zip original.  
Si no se tiene instalado kaggle, descomentar la siguiente celda y correrla. Luego entrar a la consola y correr $kaggle$. Esto creará la carpeta .kaggle en C:\Users\user\, donde se debe copiar el arcivo token (.json) que se obtiene de la subsección API en Account dentro del usuario propio de kaggle.

In [69]:
# conda install -c conda-forge kaggle

In [101]:
file = pathlib.Path("datasets/2022_03/train.csv")
if file.exists():
    print('Files already exists')
else:
    print('Creating the file')
    !kaggle competitions files -c tabular-playground-series-mar-2022
    !kaggle competitions download -c tabular-playground-series-mar-2022
    with zipfile.ZipFile("tabular-playground-series-mar-2022.zip", "r") as zip_ref:
        zip_ref.extractall(r"C:\Users\alanp\Data Science\kaggle_tabular_playground\datasets\2022_03")
    os.remove("tabular-playground-series-mar-2022.zip")

Files already exists


Luego genero los DataFrame desde los .csv utilizando pandas

In [102]:
train = pd.read_csv("datasets/2022_03/train.csv", index_col='row_id')
test = pd.read_csv("datasets/2022_03/test.csv", index_col='row_id')

In [103]:
train.head()

,time,x,y,direction,congestion
row_id,,,,,
0,1991-04-01 00:00:00,0,0,EB,70
1,1991-04-01 00:00:00,0,0,NB,49
2,1991-04-01 00:00:00,0,0,SB,24
3,1991-04-01 00:00:00,0,1,EB,18
4,1991-04-01 00:00:00,0,1,NB,60


Cantidad de mediciones por dirección al mismo tiempo en cada camino/dirección

In [104]:
train[train['time'] == '1991-05-01 00:20:00'].sort_values('direction')

,time,x,y,direction,congestion
row_id,,,,,
139945,1991-05-01 00:20:00,0,0,EB,37
139996,1991-05-01 00:20:00,2,2,EB,55
139988,1991-05-01 00:20:00,2,1,EB,93
139984,1991-05-01 00:20:00,2,0,EB,66
139978,1991-05-01 00:20:00,1,3,EB,44
...,...,...,...,...,...
139961,1991-05-01 00:20:00,0,3,WB,64
139955,1991-05-01 00:20:00,0,2,WB,34
139951,1991-05-01 00:20:00,0,1,WB,65


In [105]:
train[train['time'] == '1991-05-01 00:20:00'].groupby(['x','y','direction'])['congestion'].count().value_counts()

1    65
Name: congestion, dtype: int64

## <span style='color:#ffd700'> Feature Engineering</span>


Modifico la columna "time" y genero columnas para día de semana, horas y minutos

In [106]:
train['time'] = pd.to_datetime(train['time'])
test['time'] = pd.to_datetime(test['time'])

In [107]:
train['weekday'] = train['time'].dt.weekday
train['hour'] = train['time'].dt.hour
train['minute']  = train['time'].dt.minute
train['road'] = train['x'].astype(str) + train['y'].astype(str) + train['direction']
test['weekday'] = test['time'].dt.weekday
test['hour'] = test['time'].dt.hour
test['minute']  = test['time'].dt.minute
test['road'] = test['x'].astype(str) + test['y'].astype(str) + test['direction']

In [108]:
train.head()

,time,x,y,direction,congestion,weekday,hour,minute,road
row_id,,,,,,,,,
0,1991-04-01,0,0,EB,70,0,0,0,00EB
1,1991-04-01,0,0,NB,49,0,0,0,00NB
2,1991-04-01,0,0,SB,24,0,0,0,00SB
3,1991-04-01,0,1,EB,18,0,0,0,01EB
4,1991-04-01,0,1,NB,60,0,0,0,01NB


Genero las medianas y los desvíos estándar de congestión según el camino, hora y día

In [109]:
test

,time,x,y,direction,weekday,hour,minute,road
row_id,,,,,,,,
848835,1991-09-30 12:00:00,0,0,EB,0,12,0,00EB
848836,1991-09-30 12:00:00,0,0,NB,0,12,0,00NB
848837,1991-09-30 12:00:00,0,0,SB,0,12,0,00SB
848838,1991-09-30 12:00:00,0,1,EB,0,12,0,01EB
848839,1991-09-30 12:00:00,0,1,NB,0,12,0,01NB
...,...,...,...,...,...,...,...,...
851170,1991-09-30 23:40:00,2,3,NB,0,23,40,23NB
851171,1991-09-30 23:40:00,2,3,NE,0,23,40,23NE
851172,1991-09-30 23:40:00,2,3,SB,0,23,40,23SB


In [110]:
medians = pd.DataFrame(train.groupby(['road','weekday','hour','minute'])['congestion'].median().reset_index())
medians = medians.rename(columns={'congestion':'median'})
stds = pd.DataFrame(train.groupby(['road','weekday','hour','minute'])['congestion'].std().reset_index())
stds = stds.rename(columns={'congestion':'std'})

In [111]:
medians.sort_values('median')

,road,weekday,hour,minute,median
31824,23SW,1,0,0,11.0
31897,23SW,2,0,20,11.0
31896,23SW,2,0,0,11.0
31895,23SW,1,23,40,11.0
32123,23SW,5,3,40,11.0
...,...,...,...,...,...
29429,22WB,2,17,40,88.0
27916,22SB,2,17,20,88.0
27844,22SB,1,17,20,88.0
5477,02WB,6,1,40,88.5


In [112]:
train = train.merge(medians, how='left', on=['road','weekday','hour','minute'])
train = train.merge(stds, how='left', on=['road','weekday','hour','minute'])
X_test = test.merge(medians, how='left', on=['road','weekday','hour','minute'])
X_test = X_test.merge(stds, how='left', on=['road','weekday','hour','minute'])

In [116]:
X_test.head()

,weekday,hour,minute,road,median,std
0,0,12,0,00EB,47.0,9.240130
1,0,12,0,00NB,35.0,7.042836
2,0,12,0,00SB,56.5,10.160633
3,0,12,0,01EB,22.0,5.012139
4,0,12,0,01NB,72.0,7.441671


In [117]:
train = train.drop(columns=['time','x','y','direction'])
X_test = X_test.drop(columns=['time','x','y','direction'])

KeyError: "['time' 'x' 'y' 'direction'] not found in axis"

In [120]:
X_test

,weekday,hour,minute,road,median,std
0,0,12,0,00EB,47.0,9.240130
1,0,12,0,00NB,35.0,7.042836
2,0,12,0,00SB,56.5,10.160633
3,0,12,0,01EB,22.0,5.012139
4,0,12,0,01NB,72.0,7.441671
...,...,...,...,...,...,...
2335,0,23,40,23NB,68.0,3.314305
2336,0,23,40,23NE,25.0,15.763688
2337,0,23,40,23SB,71.0,3.425695
2338,0,23,40,23SW,11.0,9.282324


In [118]:
train

,congestion,weekday,hour,minute,road,median,std
0,70,0,0,0,00EB,35.0,11.988242
1,49,0,0,0,00NB,29.0,11.660560
2,24,0,0,0,00SB,24.0,23.050916
3,18,0,0,0,01EB,17.0,5.932911
4,60,0,0,0,01NB,63.0,4.799691
...,...,...,...,...,...,...,...
848830,54,0,11,40,23NB,58.0,2.886751
848831,28,0,11,40,23NE,26.0,2.275849
848832,68,0,11,40,23SB,71.0,4.830459
848833,17,0,11,40,23SW,20.0,10.611368


In [121]:
X = train.drop(columns=['congestion'])
y = train['congestion']
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

## <span style='color:#ffd700'> Modelado </span>


In [122]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y)

In [123]:
model = xgboost.XGBRegressor()

In [124]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [125]:
from sklearn.metrics import mean_absolute_error
pred_val = model.predict(X_val)
mean_absolute_error(y_val,pred_val)

6.074006657328761

## <span style='color:#ffd700'> Comprobación de importancia de columnas </span>

In [126]:
imp_feature = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
imp_feature

median       0.849328
road_22SW    0.020694
road_23NE    0.009222
road_00SB    0.005270
road_21SE    0.005073
               ...   
road_23SB    0.000488
road_03NB    0.000460
road_22WB    0.000395
road_21WB    0.000391
road_01SB    0.000130
Length: 70, dtype: float32

In [127]:
imp_feature.loc['weekday'], imp_feature.loc['hour'], imp_feature.loc['std']

(0.0006454694, 0.0010558421, 0.0031854468)

La mediana practicamente está definiciendo el la predición entero. Para mejorar, podría ver como dividir más los datos de tiempo. 
Se me ocurren:
- El train dataset incluye la época de vacaciones de verano en el hemisferio norte
- Horarios de tráfico laboral en días de semana
- cantidad de mediciones por hora en ese camino
- Feriados de USA, grandes eventos en USA


## <span style='color:#ffd700'> Predicción del test con el modelo elegido </span>

Primero fitteo nuevamente con el modelo completo del train dataset

In [128]:
model.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [129]:
# Habilitar cuando se tenga el modelo entrenado
pred_test = model.predict(X_test)

In [130]:
pred_test

array([47.672962, 35.216606, 55.73682 , ..., 70.962364, 16.566076,
       40.600956], dtype=float32)

In [131]:
output = pd.DataFrame({'row_id': test.index, 'congestion': pred_test})
output.to_csv('datasets/2022_03/submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [132]:
!kaggle competitions submit -c tabular-playground-series-mar-2022 -f datasets/2022_03/submission.csv -m "2- XGRegressor agregando el 'camino', la mediana de congestión y el desvío estándar"

Successfully submitted to Tabular Playground Series - Mar 2022



  0%|          | 0.00/40.0k [00:00<?, ?B/s]
 20%|#9        | 8.00k/40.0k [00:00<00:00, 39.7kB/s]
100%|##########| 40.0k/40.0k [00:02<00:00, 18.8kB/s]
